# RNN

## Import Modules

In [84]:
import torch
from torch import nn
import torchvision.datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

## Set hyper-parameters

Should notice that: in RNN for image classification, time_step could be the height of images, as:
Treat the height as the time axis for a sequence

In [85]:
epoch = 1
batch_size = 64
time_step = 28    #rnn time step/ image height
input_size = 28   #rnn input size/image width
lr =0.01  #learning rate
download_MNIST = False

## Data Acuquisiton

### train data

In [86]:
train_data = torchvision.datasets.FashionMNIST(root='C:/Users/Cesare/DeeplearningwirtPytorch/data/FashionMNIST',train=True, 
                                               transform=transforms.ToTensor(),download=False)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size,shuffle=True)

### test data

In [87]:
test_data = torchvision.datasets.FashionMNIST(root='C:/Users/Cesare/DeeplearningwirtPytorch/data/FashionMNIST',train=False, 
                                               transform=transforms.ToTensor(),download=False)
test_x =test_data.data.type(torch.FloatTensor)[:2000]/255
test_y = test_data.targets[:2000]
print(test_data.data.shape )
test_x.shape

torch.Size([10000, 28, 28])


torch.Size([2000, 28, 28])

## Build RNN

In [88]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rnn =nn.LSTM(input_size = input_size, hidden_size=64, num_layers=1,batch_first=True) #batch first设置batch和timestep的顺序
        #input size 是单行的图像的数据的长度
        #hidden_size是LSTM的隐层状态的维数：每个LSTM单元或者Timestep输出的ht的维度，
        #num_layersRNN层的个数，竖直方向上堆叠的多个相同个数单元的层数:与timestep时间轴垂直
        #------# 注意：此处rnn的return并非单个值，而是r_out, (h_n, h_c)，也就是说在传递时，也会传递这些内容
        #同时，下一层如果也是rnn，则会导入h_n 和h_c
        self.out = nn.Linear(64,10)
        
    def forward(self, x):
        #x(batch, time step,input_size)
        #r_out shape(batch, time step, out size)
        #h_n shape(n_layers, batch, hidden size)-hidden n 主线程与分线程的hidden state
        #h_c shape(n_layers, batch, hidden size)-hidden c
        r_out, (h_n, h_c) =self.rnn(x, None)   #None指的是这是第一个rnn层，因此没有前置的hidden state
        out = self.out(r_out[:,-1,:])#(batch, time_step, inputsize)此处只要最后一个time step的输出
        return out  #(batch，inputsize) 二维数组无需压缩

## Instance

In [89]:
rnn = RNN()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

## Training

In [90]:
for epoch in range(epoch):
    for step, (x,y) in enumerate(train_loader):
        x = x.view(-1,28,28) #reshape x to (batch, time_step, data width)
        output = rnn(x)
        loss = loss_func(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step%50 ==0:
            test_output = rnn(test_x)
            pred_y = torch.max(test_output,1)[1].squeeze() #此处max，dim=1读取inputsize上的最大值及index
            #获得的torch.max（test_output,1)是（max_value, index）格式。
            #使用[1]可以获得index，也就是相当于使用了argmax函数获得了内容
            #squeeze将内容压缩到二维与test_y比较
            accuracy = float(sum(pred_y ==test_y))/float(test_y.size(0))
            print('Epoch:',epoch,'|train_loss:%.4f'%loss.data.numpy(),'|test accuracy:%.2f'%accuracy)

test_output = rnn(test_x[:10].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].numpy() #转化为numpy数组打印
print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

Epoch: 0 |train_loss:2.3408 |test accuracy:0.10
Epoch: 0 |train_loss:1.3854 |test accuracy:0.50
Epoch: 0 |train_loss:1.0184 |test accuracy:0.65
Epoch: 0 |train_loss:0.7086 |test accuracy:0.70
Epoch: 0 |train_loss:0.7835 |test accuracy:0.71
Epoch: 0 |train_loss:0.7483 |test accuracy:0.75
Epoch: 0 |train_loss:0.5411 |test accuracy:0.76
Epoch: 0 |train_loss:0.7909 |test accuracy:0.78
Epoch: 0 |train_loss:0.4114 |test accuracy:0.78
Epoch: 0 |train_loss:0.6390 |test accuracy:0.80
Epoch: 0 |train_loss:0.5654 |test accuracy:0.80
Epoch: 0 |train_loss:0.4797 |test accuracy:0.81
Epoch: 0 |train_loss:0.3651 |test accuracy:0.81
Epoch: 0 |train_loss:0.5916 |test accuracy:0.81
Epoch: 0 |train_loss:0.5281 |test accuracy:0.81
Epoch: 0 |train_loss:0.3837 |test accuracy:0.84
Epoch: 0 |train_loss:0.6841 |test accuracy:0.83
Epoch: 0 |train_loss:0.4085 |test accuracy:0.84
Epoch: 0 |train_loss:0.3178 |test accuracy:0.83
[9 2 1 1 6 1 4 6 5 7] prediction number
tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7]) real numb